In [1]:
import pymysql
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sqlalchemy import create_engine

In [2]:
DORIS_HOST = 'localhost'
DORIS_PORT = 9030
DORIS_USER = 'root'
DORIS_PASSWORD = ''
DORIS_DATABASE = 'football_analytics'

In [3]:
# Méthode 1 : Connexion directe avec pymysql
def connect_doris():
    connection = pymysql.connect(
        host=DORIS_HOST,
        port=DORIS_PORT,
        user=DORIS_USER,
        password=DORIS_PASSWORD,
        database=DORIS_DATABASE,
        charset='utf8mb4'
    )
    return connection

In [ ]:
# Méthode 2 : SQLAlchemy (recommandé pour pandas)
def get_engine():
    engine = create_engine(
        f'mysql+pymysql://{DORIS_USER}:{DORIS_PASSWORD}@{DORIS_HOST}:{DORIS_PORT}/{DORIS_DATABASE}'
    )
    return engine

In [ ]:
try:
    conn = connect_doris()
    print("Connexion réussie à Apache Doris!")
    conn.close()
except Exception as e:
    print(f"Erreur de connexion : {e}")

✅ Connexion réussie à Apache Doris!


In [5]:
# Évolution de la moyenne de buts par match depuis 1950
import pandas as pd
import plotly.express as px
from sqlalchemy import create_engine

In [6]:
# Connexion
engine = connect_doris()

In [7]:
# Requête SQL
query = """
SELECT 
    YEAR(match_date) AS year,
    ROUND(AVG(home_score + away_score), 2) AS avg_goals
FROM results
WHERE YEAR(match_date) >= 1950
GROUP BY YEAR(match_date)
ORDER BY year
"""

In [9]:
df = pd.read_sql(query, engine)


C:\Users\Surface\AppData\Local\Temp\ipykernel_7404\599370298.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, engine)


In [10]:
df.head()

,year,avg_goals
0,1950,4.29
1,1951,4.27
2,1952,4.15
3,1953,3.92
4,1954,4.33


In [ ]:
import plotly.express as px
import plotly.io as pio
import pandas as pd

# Choisir le bon renderer selon ton environnement
# (décommente celui qui correspond à ton cas)
# pio.renderers.default = "notebook"   # Jupyter Notebook
pio.renderers.default = "vscode"     # VS Code
#pio.renderers.default = "browser"      # Terminal / Script .py

# Exemple de DataFrame (à remplacer par ton vrai df)
# df = pd.read_sql(query, engine)


# Création du graphique
fig = px.line(
    df,
    x='year',
    y='avg_goals',
    title='Évolution de la moyenne de buts par match (1950–2025)',
    labels={'year': 'Année', 'avg_goals': 'Moyenne de buts par match'},
    markers=True
)

# Personnalisation
fig.update_traces(
    line_color ='blue',
    line_width=3,
    marker=dict(size=6, color='green')
)

fig.update_layout(
    template='plotly_white',
    hovermode='x unified',
    font=dict(size=12),
    title_font_size=20,
    height=600
)

# Affichage
fig.show()


In [ ]:
# Chargement des données
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
#df = pd.read_sql(query, engine)

# Création du graphique interactif
fig = px.line(
    df, 
    x='year', 
    y='avg_goals',
    title='Évolution de la moyenne de buts par match (1950-2025)',
    labels={'year': 'Année', 'avg_goals': 'Moyenne de buts par match'},
    markers=True
)

# Personnalisation
fig.update_traces(
    line_color='blue',
    line_width=3,
    marker=dict(size=6, color='green')
)

fig.update_layout(
    template='plotly_white',
    hovermode='x unified',
    font=dict(size=12),
    title_font_size=20,
    height=600
)

# Affichage
fig.show()
fig.write_html('evolution_buts.html')


### Top buteurs en barres horizontales

In [12]:
# Top 20 buteurs avec détail des penalties
query_scorers = """
SELECT 
    scorer,
    team,
    COUNT(*) AS total_goals,
    SUM(CASE WHEN penalty = 1 THEN 1 ELSE 0 END) AS penalties,
    COUNT(*) - SUM(CASE WHEN penalty = 1 THEN 1 ELSE 0 END) AS goals_from_play
FROM goalscorers
WHERE (own_goal = 0 OR own_goal IS NULL)
GROUP BY scorer, team
ORDER BY total_goals DESC
LIMIT 20
"""

In [ ]:
df_scorers = pd.read_sql(query_scorers, engine)

# Création du graphique à barres horizontales empilées
fig = go.Figure()

# Buts du jeu
fig.add_trace(go.Bar(
    y=df_scorers['scorer'] + ' (' + df_scorers['team'] + ')',
    x=df_scorers['goals_from_play'],
    name='Buts du jeu',
    orientation='h',
    marker=dict(color='blue')
))

# Penalties
fig.add_trace(go.Bar(
    y=df_scorers['scorer'] + ' (' + df_scorers['team'] + ')',
    x=df_scorers['penalties'],
    name='Penalties',
    orientation='h',
    marker=dict(color='yellow')
))

fig.update_layout(
    title='Top 20 des buteurs historiques du football international',
    xaxis_title='Nombre de buts',
    yaxis_title='Joueur',
    barmode='stack',
    template='plotly_white',
    height=800,
    font=dict(size=11),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()
fig.write_html('top_buteurs.html')

C:\Users\Surface\AppData\Local\Temp\ipykernel_7404\3706740512.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



### Heatmap des confrontations entre top équipes

In [14]:
# Analyse des confrontations directes
query_heatmap = """
WITH top_teams AS (
    SELECT team, SUM(matches) as total
    FROM (
        SELECT home_team as team, COUNT(*) as matches FROM results GROUP BY home_team
        UNION ALL
        SELECT away_team as team, COUNT(*) as matches FROM results GROUP BY away_team
    ) t
    GROUP BY team
    ORDER BY total DESC
    LIMIT 15
)
SELECT 
    r.home_team,
    r.away_team,
    COUNT(*) as matches,
    SUM(CASE WHEN home_score > away_score THEN 1 ELSE 0 END) as home_wins
FROM results r
WHERE r.home_team IN (SELECT team FROM top_teams)
  AND r.away_team IN (SELECT team FROM top_teams)
GROUP BY r.home_team, r.away_team
""" 

In [15]:
df_heatmap = pd.read_sql(query_heatmap, engine)

# Création de la matrice pivot
pivot = df_heatmap.pivot(index='home_team', columns='away_team', values='matches')
pivot = pivot.fillna(0)

# Heatmap interactive
fig = px.imshow(
    pivot,
    labels=dict(x="Équipe extérieure", y="Équipe domicile", color="Nombre de matchs"),
    title="Matrice des confrontations - Top 15 équipes",
    color_continuous_scale='Blues',
    aspect="auto"
)

fig.update_layout(
    height=700,
    width=800,
    font=dict(size=10)
)

fig.show()
fig.write_html('confrontations_heatmap.html')

C:\Users\Surface\AppData\Local\Temp\ipykernel_7404\1950376226.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



### Comparatif temporel Messi vs Ronaldo

In [16]:
# Évolution des buts par année
query_goat = """
SELECT 
    YEAR(match_date) AS year,
    CASE 
        WHEN scorer LIKE '%Messi%' THEN 'Lionel Messi'
        WHEN scorer LIKE '%Ronaldo%' AND team = 'Portugal' THEN 'Cristiano Ronaldo'
    END AS player,
    COUNT(*) AS goals
FROM goalscorers
WHERE (scorer LIKE '%Messi%' AND team = 'Argentina')
   OR (scorer LIKE '%Ronaldo%' AND team = 'Portugal')
GROUP BY YEAR(match_date), player
ORDER BY year, player
"""

In [17]:
df_goat = pd.read_sql(query_goat, engine)

# Graphique comparatif
fig = px.line(
    df_goat,
    x='year',
    y='goals',
    color='player',
    title='Battle GOAT : Messi vs Ronaldo - Évolution des buts en sélection',
    labels={'year': 'Année', 'goals': 'Buts marqués', 'player': 'Joueur'},
    markers=True,
    color_discrete_map={
        'Lionel Messi': '#0066cc',
        'Cristiano Ronaldo': '#cc0000'
    }
)

fig.update_layout(
    template='plotly_white',
    hovermode='x unified',
    height=600,
    font=dict(size=12)
)

fig.show()
fig.write_html('messi_vs_ronaldo.html')

C:\Users\Surface\AppData\Local\Temp\ipykernel_7404\554745988.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



### Dashboard multi-graphiques complet

In [18]:
# Dashboard avec subplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Création de la structure du dashboard (2x2)
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Top 10 Équipes (Points/Match)',
        'Distribution des buts par période',
        'Tournois les plus représentés',
        'Évolution décennale'
    ),
    specs=[
        [{"type": "bar"}, {"type": "pie"}],
        [{"type": "bar"}, {"type": "scatter"}]
    ]
)

In [19]:
# GRAPHIQUE 1 : Top 10 équipes
query1 = """
WITH team_stats AS (
    SELECT home_team AS team,
           SUM(CASE WHEN home_score > away_score THEN 3 
                    WHEN home_score = away_score THEN 1 ELSE 0 END) AS pts,
           COUNT(*) AS matches
    FROM results WHERE match_date >= '2010-01-01'
    GROUP BY home_team
    UNION ALL
    SELECT away_team AS team,
           SUM(CASE WHEN away_score > home_score THEN 3 
                    WHEN away_score = home_score THEN 1 ELSE 0 END) AS pts,
           COUNT(*) AS matches
    FROM results WHERE match_date >= '2010-01-01'
    GROUP BY away_team
)
SELECT team, ROUND(SUM(pts)*1.0/SUM(matches), 2) as ppm
FROM team_stats
GROUP BY team
HAVING SUM(matches) >= 50
ORDER BY ppm DESC LIMIT 10
"""
df1 = pd.read_sql(query1, engine)

fig.add_trace(
    go.Bar(x=df1['team'], y=df1['ppm'], marker_color='#667eea'),
    row=1, col=1
)

# GRAPHIQUE 2 : Distribution des buts par période
query2 = """
SELECT 
    CASE 
        WHEN minute BETWEEN 1 AND 15 THEN '1-15'
        WHEN minute BETWEEN 16 AND 30 THEN '16-30'
        WHEN minute BETWEEN 31 AND 45 THEN '31-45'
        WHEN minute BETWEEN 46 AND 60 THEN '46-60'
        WHEN minute BETWEEN 61 AND 75 THEN '61-75'
        WHEN minute BETWEEN 76 AND 90 THEN '76-90'
        ELSE '90+'
    END AS period,
    COUNT(*) AS goals
FROM goalscorers
WHERE minute IS NOT NULL
GROUP BY period
"""
df2 = pd.read_sql(query2, engine)

fig.add_trace(
    go.Pie(labels=df2['period'], values=df2['goals'], marker_colors=px.colors.sequential.Blues),
    row=1, col=2
)

# GRAPHIQUE 3 : Tournois populaires
query3 = """
SELECT tournament, COUNT(*) as matches
FROM results
GROUP BY tournament
ORDER BY matches DESC
LIMIT 8
"""
df3 = pd.read_sql(query3, engine)

fig.add_trace(
    go.Bar(x=df3['matches'], y=df3['tournament'], orientation='h', marker_color='#764ba2'),
    row=2, col=1
)

# GRAPHIQUE 4 : Évolution décennale
query4 = """
SELECT 
    FLOOR(YEAR(match_date)/10)*10 AS decade,
    ROUND(AVG(home_score + away_score), 2) AS avg_goals
FROM results
WHERE YEAR(match_date) >= 1950
GROUP BY decade
ORDER BY decade
"""
df4 = pd.read_sql(query4, engine)

fig.add_trace(
    go.Scatter(x=df4['decade'], y=df4['avg_goals'], mode='lines+markers', 
               line=dict(color='#ffc107', width=3)),
    row=2, col=2
)

# Mise en page globale
fig.update_layout(
    title_text="Dashboard Analytique - Football International",
    showlegend=False,
    height=900,
    template='plotly_white'
)

fig.show()
fig.write_html('dashboard_complet.html')

C:\Users\Surface\AppData\Local\Temp\ipykernel_7404\2316694568.py:24: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\Surface\AppData\Local\Temp\ipykernel_7404\2316694568.py:48: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\Surface\AppData\Local\Temp\ipykernel_7404\2316694568.py:63: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\Surface\AppData\Local\Temp\ipykernel_7404\2316694568.py:80: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBA